In [41]:
import math
import matplotlib.pyplot as plt

class Value:
    def __init__(self, data, _children=(), _op='', label = ''):
        self.data = data
        self._prev = _children
        self.grad = 0.0
        self._op = _op
        self._label = label
        self._backward = lambda: None

    def __repr__(self):
        return f'Value: {self.data}'
    
    def __add__(self, other):
        self = self if isinstance(self, Value) else Value(self)
        other = other if isinstance(other, Value) else Value(other)
        
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        self._backward = _backward
        return out
    
    def __mul__(self, other):
        out =  Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        self._backward = _backward
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += 1 - self.tanh()**2
        self._backward = _backward
        return out

    

    def backprop(node):
        # Topo sort
        topo = []
        visited = set()
        def build_topo(v):
            for child in v._prev:
                if child not in visited:
                    visited.add(child)
                    build_topo(child)
            topo.append(v)
        build_topo(node)
        for node in reversed(topo):
            node._backward()

        for node in reversed(topo):
            print(f'Label: {node._label}, value: {node.data}, grad: {node.grad}')

    def zero_grad(self):
        if not self._prev:
            return 
        
        self.grad = 0
        for node in self._prev:
            node.zero_grad()

a = Value(3, label='a')
b = Value(4, label='b')
c = Value(-2, label='c')
d = a + 1
e = d + a
f = e + 4
L = f * c
d._label = 'd'
e._label = 'e'
f._label = 'f'
L._label = 'L'
L.grad = 1




Label: L, value: -22, grad: 1
Label: c, value: -2, grad: 11.0
Label: f, value: 11, grad: -2.0
Label: , value: 4, grad: -2.0
Label: e, value: 7, grad: -2.0
Label: d, value: 4, grad: -2.0
Label: , value: 1, grad: -2.0
Label: a, value: 3, grad: -4.0


In [151]:
L.zero_grad()
L.grad = 1
L.backprop()
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
d.data += 0.01 * d.grad
e.data += 0.01 * e.grad
f.data += 0.01 * f.grad
print("L_data: ", f.data * c.data)

Label: L, value: -22, grad: 1
Label: c, value: 2676.8378107448143, grad: 12595.277282196055
Label: f, value: 567.2516866401859, grad: 2676.8378107448143
Label: , value: 4, grad: 58300.0064747634
Label: e, value: 563.2516866401859, grad: 2676.8378107448143
Label: d, value: 560.2516866401858, grad: 2676.8378107448143
Label: , value: 1, grad: 58300.0064747634
Label: a, value: 22747.19456439211, grad: 116600.0129495268
L_data:  1664913.8439243946


In [582]:
print(d.grad)

-2.0
